In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN-LSTM4003"

## Running CNN then RNN
5 layers CNN [2, 2, 2, 2, 2], then RNN, bias_shape = [32,64,64, 64, 64]

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import sys
from hdf5storage import savemat
import pickle

sys.path.append('../../../src')
sys.path.append('../../../src/models')

In [3]:
import models.CNN_lstm_model as CNN_lstm_model
import training  
import evaluation.nn_eval as nn_eval 

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

## Model

In [5]:
## Model Parameter
n_variables= 12
learning_rate = 0.001
batch_size = 10

pooling_stride = [32, 1, 1, 1, 1, 1]
weights_shape = [[5,n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32],
                 [5, 32, 32],
                 [5, 32, 32],
                 [5, 32, 32]]

bias_shape = [32, 32, 32, 32, 32, 32, 32]
activation = tf.nn.relu

lstm_n_hidden = 128
n_classes = 1


#positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [6]:
pos_weight

3.186018221237171

In [7]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [8]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 


In [9]:
model = CNN_lstm_model.Model(n_variables, n_classes)

In [10]:
model.build_cnn_lstm_net(pooling_stride,
                         weights_shape,
                         bias_shape,
                         activation,
                         lstm_n_hidden,
                         n_classes,
                         learning_rate,
                         pos_weight)

In [11]:
train_obj = training.Trainer(model,
                 train_data, train_gt,
                 validation_data, validation_gt,
                 sess,
                 summary_path,
                 batch_size,
                 global_step = 0)

In [12]:
train_obj.train(max_iter = 10000,
                train_eval_step=100,
                validation_eval_step=100,
                display_step=100)

Tarin Iter 0, auc= 0.304253, fmeasure_max= 0.422757, precision= 0.278294, recall= 0.879102
Train Random guess:  , auc= 0.224143fmeasure_max= 0.370157, precision= 0.227201, recall= 0.998273
Validation Iter 0, auc= 0.218760, fmeasure_max= 0.421053, precision= 0.271287, recall= 0.939966
Validation Random guess:  auc= 0.234939, fmeasure_max= 0.371101, precision= 0.227823, recall= 1.000000
Minibatch Loss= 1.122007
Tarin Iter 100, auc= 0.689156, fmeasure_max= 0.667186, precision= 0.769784, recall= 0.588721
Train Random guess:  , auc= 0.273738fmeasure_max= 0.442618, precision= 0.284206, recall= 1.000000
Validation Iter 100, auc= 0.617026, fmeasure_max= 0.624754, precision= 0.731034, recall= 0.545455
Validation Random guess:  auc= 0.248185, fmeasure_max= 0.370983, precision= 0.227734, recall= 1.000000
Minibatch Loss= 0.953038
Tarin Iter 200, auc= 0.804546, fmeasure_max= 0.753668, precision= 0.761310, recall= 0.746177
Train Random guess:  , auc= 0.258328fmeasure_max= 0.407096, precision= 0.2555

In [13]:
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=train_obj.global_step)

ValueError: Parent directory of ../model/CNN-LSTM4003/CNN-LSTM4003 doesn't exist, can't save.

In [ ]:
saver = tf.train.Saver()
saver.restore(sess, checkpoint_path+'-1000')

In [14]:
probability_of_pos = nn_eval.evaluate_model(model,
                                   sess,
                                   test_data,
                                   test_gt)

Testing auc = 0.901307 F1_max = 0.826237 prec = 0.843541 rec = 0.809628
Test Random guess: auc= 0.227852 fmeasure_max= 0.375723 , precision= 0.231362 , recall= 0.999155


In [15]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [16]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})